# Generating recipes using pretrained GPT-2 Model
* see documentation [here](https://huggingface.co/docs/transformers/en/model_doc/gpt2)

### With time and ingredients

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed

# Set a random seed for reproducibility
set_seed(42)

# Load pre-trained GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set the pad token to the EOS token if it's not already defined
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name, pad_token_id=tokenizer.eos_token_id)

# Define ingredients and create a prompt
ingredients = "chicken, garlic, olive oil, basil"
max_min = 120
prompt = f"Given that you have {max_min} minutes to cook, here is a recipe using the following ingredients: {ingredients}. First, you should"

# Encode the prompt, ensuring to add attention mask and padding
inputs = tokenizer(prompt, return_tensors="pt", padding="max_length", max_length=100, truncation=True)
attention_mask = inputs['attention_mask']

# Generate response using the encoded inputs and attention mask
output = model.generate(
    input_ids=inputs['input_ids'],
    attention_mask=attention_mask,
    max_length=200,
    num_beams=5,
    no_repeat_ngram_size=2,
    early_stopping=True
)

# Decode and print the generated recipe
generated_recipe1 = tokenizer.decode(output[0], skip_special_tokens=True)
generated_recipe1.split(".")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


['Given that you have 120 minutes to cook, here is a recipe using the following ingredients: chicken, garlic, olive oil, basil',
 ' First, you should\n\ncook the chicken in a large skillet over medium-high heat',
 ' Add the garlic and cook for about 5 minutes, or until the onion is translucent',
 ' Remove from the heat and allow to cool slightly',
 ' Once the onions are translucent, add the basil and continue cooking for another 5-10 minutes or so until they are soft and tender, about 3-4 minutes per side',
 ' Stir in the tomato paste, salt, pepper, and cayenne pepper',
 ' Serve immediately or refrigerate',
 '\n']

In [2]:
generated_recipe1.split(".")

['Given that you have 120 minutes to cook, here is a recipe using the following ingredients: chicken, garlic, olive oil, basil',
 ' First, you should\n\ncook the chicken in a large skillet over medium-high heat',
 ' Add the garlic and cook for about 5 minutes, or until the onion is translucent',
 ' Remove from the heat and allow to cool slightly',
 ' Once the onions are translucent, add the basil and continue cooking for another 5-10 minutes or so until they are soft and tender, about 3-4 minutes per side',
 ' Stir in the tomato paste, salt, pepper, and cayenne pepper',
 ' Serve immediately or refrigerate',
 '\n']

In [3]:
# Define your ingredients and create a prompt
ingredients = "chicken, garlic, olive oil, basil"
prompt = f"Recipe:\nIngredients: {ingredients}\nInstructions:\n"

# Encode the prompt, ensuring to add attention mask and padding
inputs = tokenizer(prompt, return_tensors="pt", padding="max_length", max_length=150, truncation=True)
attention_mask = inputs['attention_mask']

# Generate response using the encoded inputs and attention mask
output = model.generate(
    input_ids=inputs['input_ids'],
    attention_mask=attention_mask,
    max_length=250,
    num_beams=5,
    no_repeat_ngram_size=2,
    early_stopping=True,
    temperature=0.9,
    eos_token_id=tokenizer.eos_token_id
)

# Decode and print the generated recipe
generated_recipe2 = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_recipe2)

/Users/jadeliang/anaconda3/envs/pic16B/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Recipe:
Ingredients: chicken, garlic, olive oil, basil
Instructions:
1. Preheat oven to 350 degrees F.
2. In a large bowl, whisk together the chicken and garlic. Stir in the basil and cook until fragrant, about 5 minutes. Remove from heat and set aside. 3. Heat the oil in a skillet over medium-high heat. Add the onion and sauté for about 2 minutes, stirring occasionally, until soft and translucent. Transfer to a wire rack and let cool completely. 4. Once the onions are translucent, add the


In [4]:
generated_recipe2.split(".")

['Recipe:\nIngredients: chicken, garlic, olive oil, basil\nInstructions:\n1',
 ' Preheat oven to 350 degrees F',
 '\n2',
 ' In a large bowl, whisk together the chicken and garlic',
 ' Stir in the basil and cook until fragrant, about 5 minutes',
 ' Remove from heat and set aside',
 ' 3',
 ' Heat the oil in a skillet over medium-high heat',
 ' Add the onion and sauté for about 2 minutes, stirring occasionally, until soft and translucent',
 ' Transfer to a wire rack and let cool completely',
 ' 4',
 ' Once the onions are translucent, add the']

## Training GPT2 Model Using Scraped Recipes

In [5]:
import pandas as pd
recipes = pd.read_csv("cleaned_recipes.csv")
recipes.head()

,recipe_name,category_name,rating,prep_time,cook_time,additional_time,total_time,num_servings_per_recipe,ingredients_list,direction_list,...,carbs (g),fiber (g),sugar (g),cholesterol (mg),vitamin_c (mg),calcium (mg),iron (mg),potassium (mg),recipe_link,main_ingredients
0,Air Fryer Spicy Onion Rings,Air Fryer Recipes,5.0,20.0,10.0,30.0,60.0,4,"sweet onions, sliced 1/2 inch thick,buttermilk...","Whisk together buttermilk, egg, flour, chile ...",...,53.0,2.0,NaN,48.0,NaN,NaN,NaN,197.0,https://www.allrecipes.com/recipe/8465728/air-...,"['bread', 'egg', 'flour', 'lime', 'milk', 'oni..."
1,Stuffed Chicken Cordon Bleu,Chicken Cordon Bleu,5.0,20.0,40.0,15.0,75.0,4,"skinless, boneless chicken breast halves,bacon...",Preheat the oven to 400 degrees F (200 degree...,...,44.0,3.0,8.0,291.0,3.0,636.0,4.0,969.0,https://www.allrecipes.com/recipe/283793/stuff...,"['apple', 'bacon', 'bread', 'cheese', 'chicken..."
2,Hearty Chicken Cacciatore Soup with Rice,Chicken Cacciatore,NaN,10.0,105.0,0.0,115.0,10,"chicken broth,condensed tomato soup,water,dice...","Combine chicken broth, condensed soup, 2 cans...",...,35.0,2.0,NaN,38.0,NaN,NaN,NaN,406.0,https://www.allrecipes.com/recipe/8300735/hear...,"['chicken', 'mushroom', 'onion', 'rice', 'toma..."
3,Chicken and Dumplings with Biscuits,Chicken and Dumplings,4.1,NaN,NaN,NaN,NaN,8,"whole chicken, cut into pieces,salt,freshly gr...",Put chicken pieces in a large pot over medium...,...,86.0,6.0,5.0,106.0,49.0,64.0,5.0,1398.0,https://www.allrecipes.com/recipe/8810/chicken...,"['carrot', 'chicken', 'flour', 'potato']"
4,Margo's Chicken Adobo,Chicken Adobo,4.5,10.0,60.0,0.0,70.0,8,"canola oil,chicken drumsticks and thighs,onion...",Heat canola oil in a large Dutch oven over me...,...,7.0,1.0,2.0,96.0,3.0,40.0,3.0,347.0,https://www.allrecipes.com/recipe/218510/margo...,"['apple', 'chicken', 'corn', 'onion']"


In [6]:
recipes["direction_list"][0].strip()

'Whisk together buttermilk, egg, flour, chile and lime seasoning, and adobo seasoning for the batter in a shallow bowl. Cover and refrigerate for 30 minutes.\n Combine panko, adobo seasoning, and chile and lime seasoning in a shallow dish; mix well. Remove batter from the fridge. Dip onion rings first into the batter, then into bread crumb mixture, turning to coat, and gently shake off excess crumbs. Lightly spritz the onion rings with cooking spray on both sides.\n Preheat the air fryer to 340 degrees F (170 degrees C). Line the air fryer basket with a parchment liner or lightly spray with oil.\n Place the breaded onion rings into the fryer basket in an even layer, leaving about 1/2-inch space between the slices.\n Cook until crisp and lightly browned, flipping halfway through, 10 to 12 minutes. You may have to cook in batches, and cooking time may vary depending on the size and brand of your air fryer.\n Remove from the air fryer, transfer to a baking sheet, sprinkle with kosher salt

### Preparing values in `recipes` dataframe

In [7]:
# converting values in "main_ingredients" column into lists
def clean_main_ingredients(ingredient_string):
    """ Converts main_ingredients string into a list of ingredients"""
    return ingredient_string.replace("[", "").replace("]", "").replace("'", "").split(",")

recipes["main_ingredients"] = recipes["main_ingredients"].apply(clean_main_ingredients)
recipes["main_ingredients"].head()

0          [bread,  egg,  flour,  lime,  milk,  onion]
1    [apple,  bacon,  bread,  cheese,  chicken,  co...
2       [chicken,  mushroom,  onion,  rice,  tomatoes]
3                  [carrot,  chicken,  flour,  potato]
4                     [apple,  chicken,  corn,  onion]
Name: main_ingredients, dtype: object

In [8]:
# converting values in "directions" into lists
def clean_directions(direction_string):
    """ Converts direction_string into a list of directions"""
    return direction_string.replace("\n", '').split(". ")

recipes["direction_list"] = recipes["direction_list"].apply(clean_directions)
recipes["direction_list"].head()

0    [ Whisk together buttermilk, egg, flour, chile...
1    [ Preheat the oven to 400 degrees F (200 degre...
2    [ Combine chicken broth, condensed soup, 2 can...
3    [ Put chicken pieces in a large pot over mediu...
4    [ Heat canola oil in a large Dutch oven over m...
Name: direction_list, dtype: object

In [9]:
recipes["total_time"] = recipes["total_time"].apply(lambda x: [x])
recipes["total_time"].head()

0     [60.0]
1     [75.0]
2    [115.0]
3      [nan]
4     [70.0]
Name: total_time, dtype: object

In [10]:
# takes in each row in recipes df and formats it into
# a string that starts and ends with special tokens 
# [RECIPE_START] and [RECIPE_END], which helps
# model recognize the beginning and end of each recipe
recipes['formatted_text'] = recipes.apply(lambda row: f"[RECIPE_START] Ingredients: {row['main_ingredients']} Total Time: {row['total_time']} minutes Directions: {row['direction_list']} [RECIPE_END]", axis=1)
recipes['formatted_text'].head()

0    [RECIPE_START] Ingredients: ['bread', ' egg', ...
1    [RECIPE_START] Ingredients: ['apple', ' bacon'...
2    [RECIPE_START] Ingredients: ['chicken', ' mush...
3    [RECIPE_START] Ingredients: ['carrot', ' chick...
4    [RECIPE_START] Ingredients: ['apple', ' chicke...
Name: formatted_text, dtype: object

In [11]:
recipes = recipes[["main_ingredients", "total_time", "direction_list", "formatted_text"]]
recipes.rename(columns={'main_ingredients': 'ingredients',
                        'direction_list': 'directions'}, inplace=True)

In [12]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
import pandas as pd
from datasets import Dataset

# Load the tokenizer and the model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Set the EOS token as the padding token
tokenizer.pad_token = tokenizer.eos_token

# Resize the token embeddings 
model.resize_token_embeddings(len(tokenizer))

# Convert DataFrame to Hugging Face dataset
hf_dataset = Dataset.from_pandas(recipes)

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['formatted_text'], truncation=True, padding="max_length", max_length=512)

tokenized_data = hf_dataset.map(tokenize_function, batched=True)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Map:   0%|          | 0/14480 [00:00<?, ? examples/s]

### Attempt to find tune GPT-2 model using scraped recipes....
* haven't run the following code to completion (took 7 mins to complete 1% training; made computer very laggy)

In [14]:
# fine-tune GPT-2 model
from datasets import Dataset, DatasetDict
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments

# split data set into train and test
split_datasets = tokenized_data.train_test_split(test_size=0.1) 
dataset_dict = DatasetDict({
    'train': split_datasets['train'],
    'test': split_datasets['test']
})

def add_labels(examples):
    # The labels are the same as the input_ids for language modeling tasks
    examples['labels'] = examples['input_ids'].copy()
    return examples

# Apply the function to add labels
dataset_dict = dataset_dict.map(add_labels)

# Resize model embeddings to accommodate new tokens
model.resize_token_embeddings(len(tokenizer))

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=2,
    save_steps=500,
    save_total_limit=2,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    dataloader_num_workers=4
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_dict['train'],
    eval_dataset=dataset_dict['test']
)

# Start training
trainer.train()

Map:   0%|          | 0/13032 [00:00<?, ? examples/s]

Map:   0%|          | 0/1448 [00:00<?, ? examples/s]

  0%|          | 0/9774 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [ ]:
trainer.save_model("./fine_tuned_recipe_model")
tokenizer.save_pretrained("./fine_tuned_recipe_model")

In [ ]:
# Load the model and tokenizer
model_tuned = GPT2LMHeadModel.from_pretrained("./fine_tuned_recipe_model")
tokenizer = GPT2Tokenizer.from_pretrained("./fine_tuned_recipe_model")

In [ ]:
# Generate response using the encoded inputs and attention mask
output_tuned = model_tuned.generate(
    input_ids=inputs['input_ids'],
    attention_mask=attention_mask,
    max_length=250,
    num_beams=5,
    no_repeat_ngram_size=2,
    early_stopping=True,
    temperature=0.9,
    eos_token_id=tokenizer.eos_token_id
)

# Decode and print the generated recipe
generated_recipe2 = tokenizer.decode(output_tuned[0], skip_special_tokens=True)
print(generated_recipe2)